In [1]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import scipy
from sympy.physics.mechanics import *
from scipy import linalg


In [2]:
o_x, o_z, o_x_dot, o_z_dot, theta, theta_dot, alpha, alpha_dot, f_z, tau_y = sym.symbols(r'o_x, o_z, o_x_dot, o_z_dot, theta, theta_dot, alpha, alpha_dot, f_z, tau_y')
m, J_x, J_y, J_z, g, m_pen, l_pen = sym.symbols('m, J_x, J_y, J_z, g, m_pen, l_pen')
dyn_o_x, dyn_o_z, dyn_alpha, dyn_theta = dynamicsymbols(r'o_x o_z \alpha \theta')
dyn_o_xd, dyn_o_zd, dyn_alphad, dyn_thetad = dynamicsymbols(r'o_x o_z \alpha \theta', 1)
dyn_o_xdd, dyn_o_zdd, dyn_alphadd, dyn_thetadd = dynamicsymbols(r'o_x o_z \alpha \theta', 2)
L = 1/2*m*(dyn_o_xd**2 + dyn_o_zd**2) + 1/2*J_y*(dyn_thetad**2) + 1/2*m_pen*(dyn_o_xd**2 + dyn_o_zd**2 + l_pen**2 * dyn_alphad**2 + 2*l_pen*dyn_alphad*(dyn_o_xd*sym.cos(dyn_alpha) - dyn_o_zd*sym.sin(dyn_alpha))) - m*g*dyn_o_z - m_pen*g*(dyn_o_z + l_pen*sym.cos(dyn_alpha))
dyn_f_z, dyn_tau_y = sym.symbols(r'f_z, \tau_y')
t = sym.Symbol('t')

Equations = sym.Matrix([[sym.simplify(sym.diff(sym.diff(L, dyn_o_xd), t) - sym.diff(L, dyn_o_x) - dyn_f_z*sym.sin(dyn_theta))],
             [sym.simplify(sym.diff(sym.diff(L, dyn_o_zd), t) - sym.diff(L, dyn_o_z) - dyn_f_z*sym.cos(dyn_theta))],
             [sym.simplify(sym.diff(sym.diff(L, dyn_alphad), t) - sym.diff(L, dyn_alpha))],
             [sym.simplify(sym.diff(sym.diff(L, dyn_thetad), t) - sym.diff(L, dyn_theta) - dyn_tau_y)]])


In [3]:
Equations[3,0]

1.0*J_y*Derivative(\theta(t), (t, 2)) - \tau_y

In [4]:
ox_ddot_with_alphaddot = sym.solve(Equations[0,0], dyn_o_xdd)[0]
oz_ddot_with_alphaddot = sym.solve(Equations[1,0], dyn_o_zdd)[0]
alpha_ddot_eqn = Equations[2].subs([(dyn_o_xdd, ox_ddot_with_alphaddot), (dyn_o_zdd, oz_ddot_with_alphaddot)])
alpha_ddot_soln = sym.simplify(sym.solve(alpha_ddot_eqn, dyn_alphadd)[0])
new_ox_ddot = sym.simplify(ox_ddot_with_alphaddot.subs(dyn_alphadd, alpha_ddot_soln))
new_oz_ddot = sym.simplify(oz_ddot_with_alphaddot.subs(dyn_alphadd, alpha_ddot_soln))
theta_ddot_soln = sym.solve(Equations[3,0], dyn_thetadd)[0]
theta_ddot_soln = theta_ddot_soln.subs(dyn_tau_y, tau_y)
new_ox_ddot = new_ox_ddot.subs([(dyn_f_z, f_z), (dyn_theta, theta), (dyn_alphad, alpha_dot)])
new_ox_ddot = new_ox_ddot.subs(dyn_alpha, alpha)
new_oz_ddot = new_oz_ddot.subs([(dyn_f_z, f_z), (dyn_theta, theta), (dyn_alphad, alpha_dot)])
new_oz_ddot = new_oz_ddot.subs(dyn_alpha, alpha)
alpha_ddot_soln = alpha_ddot_soln.subs([(dyn_alpha, alpha), (dyn_theta, theta), (dyn_f_z, f_z)])


In [5]:
f_sym_planar = sym.Matrix.vstack(sym.Matrix([o_x_dot]),
                          sym.Matrix([new_ox_ddot]),
                          sym.Matrix([o_z_dot]),
                          sym.Matrix([new_oz_ddot]),
                          sym.Matrix([theta_dot]),
                          sym.Matrix([theta_ddot_soln]),
                          sym.Matrix([alpha_dot]),
                          sym.Matrix([alpha_ddot_soln])
                          )
f_sym_planar

Matrix([
[                                                                                                                             o_x_dot],
[               (-f_z*m_pen*sin(alpha - theta)*cos(alpha) + m*(alpha_dot**2*l_pen*m_pen*sin(alpha) + f_z*sin(theta)))/(m*(m + m_pen))],
[                                                                                                                             o_z_dot],
[(f_z*m_pen*sin(alpha)*sin(alpha - theta) + m*(alpha_dot**2*l_pen*m_pen*cos(alpha) + f_z*cos(theta) - g*m - g*m_pen))/(m*(m + m_pen))],
[                                                                                                                           theta_dot],
[                                                                                                                           tau_y/J_y],
[                                                                                                                           alpha_dot],
[                                      

In [6]:
s_planar = [o_x, o_x_dot, o_z, o_z_dot, theta, theta_dot, alpha, alpha_dot]
i_planar = [tau_y, f_z]
p_planar = [m, J_y, g, m_pen, l_pen]
s_with_des_planar = [o_x, o_z]

In [7]:
f_planar = sym.lambdify(s_planar + i_planar + p_planar, f_sym_planar)

In [8]:
f_planar

<function _lambdifygenerated(o_x, o_x_dot, o_z, o_z_dot, theta, theta_dot, alpha, alpha_dot, tau_y, f_z, m, J_y, g, m_pen, l_pen)>

In [9]:
# Mass
m = 0.044

# Principle moments of inertia
J_x = 1.57e-05
J_y = 1.60e-05
J_z = 2.05e-05

# Acceleration of gravity
g = 9.81

# Pendulum parameters
l_pen = 150e-3 # maybe m
m_pen = 0.006

In [10]:
p_planar_eq = [m, J_y, g, m_pen, l_pen]
s_planar_eq = [0., 0., 0., 0., 0., 0., 0., 0.]
i_planar_eq = [0., g*(m + m_pen)]

In [11]:
print(f_planar(*s_planar_eq, *i_planar_eq, *p_planar_eq))

[[ 0.0000000e+00]
 [ 0.0000000e+00]
 [ 0.0000000e+00]
 [-6.9388939e-16]
 [ 0.0000000e+00]
 [ 0.0000000e+00]
 [ 0.0000000e+00]
 [ 0.0000000e+00]]


In [12]:
A_planar_sym = f_sym_planar.jacobian(s_planar)
B_planar_sym = f_sym_planar.jacobian(i_planar)

In [13]:
A_planar_num = sym.lambdify(s_planar + i_planar + p_planar, A_planar_sym)
B_planar_num = sym.lambdify(s_planar + i_planar + p_planar, B_planar_sym)

In [14]:
A_planar = A_planar_num(*s_planar_eq, *i_planar_eq, *p_planar_eq)
B_planar = B_planar_num(*s_planar_eq, *i_planar_eq, *p_planar_eq)

In [15]:
A_planar_str = np.array2string(A_planar,
                        formatter={'float_kind': lambda x: f'{x:5.2f}'},
                        prefix='    ',
                        max_line_width=np.inf)

print(f'A = {A_planar_str}')

A = [[ 0.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00 11.15  0.00 -1.34  0.00]
     [ 0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00 -0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00]
     [ 0.00  0.00  0.00  0.00 -74.32  0.00 74.32  0.00]]


In [16]:
B_planar_str = np.array2string(B_planar,
                        formatter={'float_kind': lambda x: f'{x:10.2f}'},
                        prefix='    ',
                        max_line_width=np.inf)
print(f'B = {B_planar_str}')

B = [[      0.00       0.00]
     [      0.00       0.00]
     [      0.00       0.00]
     [      0.00      20.00]
     [      0.00       0.00]
     [  62500.00       0.00]
     [      0.00       0.00]
     [      0.00       0.00]]


In [17]:
W = np.block([[B_planar, A_planar @ B_planar, A_planar @ A_planar @ B_planar, np.linalg.matrix_power(A_planar, 3) @ B_planar, np.linalg.matrix_power(A_planar, 4) @ B_planar, np.linalg.matrix_power(A_planar, 5) @ B_planar, np.linalg.matrix_power(A_planar, 6) @ B_planar, np.linalg.matrix_power(A_planar, 7) @ B_planar]])

np.linalg.matrix_rank(W) # If 8, the system is controllable
linalg.svdvals(W)

array([2.56610787e+10, 2.56610787e+10, 6.13025688e+05, 6.13025688e+05,
       6.25000000e+04, 6.25000000e+04, 2.00000000e+01, 2.00000000e+01])

In [18]:
# Controller Design
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K

In [19]:
Q = np.diag([
    1., # x
    1., #x_dot
    1., # z
    1., # z_dot
    1., # theta (pitch)
    1., # theta_dot
    1., #alpha
    1. #alpha_dot
])

R = np.diag([
    10E6,#tau_y
    10, #f_z
])

In [26]:
K = lqr(A_planar, B_planar, Q, R)
K_str = np.array2string(K,
                        formatter={'float_kind': lambda x: f'{x:6.3f}'},
                        prefix='    ',
                        max_line_width=np.inf)

K

array([[-3.16227766e-04, -5.62372285e-04, -3.75118589e-18,
        -3.58037631e-18,  1.18799716e-02,  6.92935127e-04,
        -1.52344145e-02, -1.80434720e-03],
       [ 2.70362764e-15,  2.30331138e-15,  3.16227766e-01,
         3.62798534e-01, -4.13236788e-14, -1.14572042e-15,
         6.80987900e-14,  7.57825355e-15]])